In [3]:
from dotenv import load_dotenv
import os
import cassiopeia as cass

In [5]:
load_dotenv()

API_KEY = os.getenv("API_KEY")

#API_KEY="Uncomment this line and put your API key file, if you're not using an .env file"

cass.set_riot_api_key(API_KEY) 

In [33]:
patch = cass.Patch.latest(region="EUW")

patch.name

'13.24'

In [37]:
summoner = cass.get_summoner(name="IntensiveImpact", region="EUW")

match = summoner.match_history[0]

Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/joSl8LsJyQhK55zztnuO4bwduaW_FPo4mxOkRu2Syn5f-WoJ1ECyllOfAlSXsQJarztC3MuewBLPnQ/ids?start=0&count=100


In [99]:
[ (x.id, x.summoner.id, x.summoner.name, x.champion.name) for x in match.participants ]

Making call: https://ddragon.leagueoflegends.com/cdn/13.24.1/data/en_GB/championFull.json
Making call: https://ddragon.leagueoflegends.com/realms/euw.json


[(1,
  'ePLL3pJs1TlSIbADsNrPFrq8IkaP0NNf_4MJ0dqBmAbwTo8gvBLnpjQd7w',
  'IntensiveImpact',
  'Gnar'),
 (2,
  'ovaHtvqojIzvmSPVg614eCXxCNCIPZlxdCcP2MSX3gk-tLbfKlqans8zNA',
  'GG Garen Gaming',
  'Udyr'),
 (3, 'C55E7_u1wsCFC8dRuu8uS0TPrw7PwWAoka9xYZiZyM02xqo', 'Candywood', 'Veigar'),
 (4, 'jM2q7SIckc01URc9mr5lYGhJ-iLuprfC7DL7EbDikPX6jak', 'Zhen Hu', 'Brand'),
 (5, 'i_40UuHmSn0kK5XIvtJZTC5yoh75lPDTprAuc4SrSl4ygN8', 'Vanílla', 'Pantheon'),
 (6,
  '8iPBoRJOO3OTMOHBPTdkwVacJve9fWv8SFpWOwmnYZf8sB8',
  'MastersGaminG',
  'Illaoi'),
 (7,
  'HuEeFBh4pZU8at5zZPiMTSWp3RY9sW4NVFzBjsadmQqqoctp0Wt8kdkLMw',
  'Legoooshi',
  'Lillia'),
 (8,
  'eAOQRz5IP72JCnJAqLT1GTQwTgWATLZSVQkXs-iBDT7JHpbk5IEZpCQL1w',
  'orgengianni',
  'Nasus'),
 (9,
  'NXK_Jc44Sfa_BAdbOxpBzJOtb2KL77w5v5yPeA7_H1Ek6bWP14hB6nxF1Q',
  'Hsagul',
  'Volibear'),
 (10,
  '2kzIYUTbnGZ17G6dn6X-87fG2Vcv81IaaEbxKfsMfY8Y9QmB1y3I5UVZvA',
  'Phiolyeine',
  'Ashe')]

In [104]:
participant_frames = sum([ [(participant_frame, frame.timestamp) for participant_frame in list(frame.participant_frames.values())] for frame in match.timeline.frames ], [])
positions = [
    (
     participant_frame[0].participant_id,
     ( 
        participant_frame[0].position.x,
        participant_frame[0].position.y
     ),
     int(participant_frame[1].seconds * 1000 + participant_frame[1].microseconds / 1000),
     participant_frame[0].position.location.value
    ) for participant_frame in participant_frames
]
positions

[(1, (554, 581), 0, 'NEXUS_BLUE'),
 (2, (593, 464), 0, 'NEXUS_BLUE'),
 (3, (557, 345), 0, 'NEXUS_BLUE'),
 (4, (458, 271), 0, 'NEXUS_BLUE'),
 (5, (335, 269), 0, 'NEXUS_BLUE'),
 (6, (14500, 14511), 0, 'NEXUS_RED'),
 (7, (14539, 14394), 0, 'NEXUS_RED'),
 (8, (14503, 14275), 0, 'NEXUS_RED'),
 (9, (14404, 14201), 0, 'NEXUS_RED'),
 (10, (14281, 14199), 0, 'NEXUS_RED'),
 (1, (1024, 3759), 60028, 'NEXUS_BLUE'),
 (2, (7150, 3132), 60028, 'JUNGLE_BOT_BLUE'),
 (3, (5620, 6200), 60028, 'MID_LANE_BLUE'),
 (4, (7026, 3118), 60028, 'JUNGLE_BOT_BLUE'),
 (5, (10278, 3002), 60028, 'JUNGLE_BOT_BLUE'),
 (6, (6674, 11356), 60028, 'JUNGLE_TOP_RED'),
 (7, (11519, 7501), 60028, 'JUNGLE_BOT_RED'),
 (8, (8568, 6494), 60028, 'MID_LANE_PURPLE'),
 (9, (13440, 4191), 60028, 'BOT_LANE_RED'),
 (10, (13729, 3876), 60028, 'BOT_LANE_RED'),
 (1, (874, 11306), 120047, 'TOP_LANE_BLUE'),
 (2, (8523, 2455), 120047, 'JUNGLE_BOT_BLUE'),
 (3, (6747, 7253), 120047, 'NONE'),
 (4, (10932, 1471), 120047, 'BOT_LANE_BLUE'),
 (5, (108

In [111]:
import numpy as np

output_file = "api_test.npz"

classes = []
points = []
timestamps = []
for class_label, point, timestamp, _ in positions:
    classes.append(int(class_label)-1)
    points.append(point)
    timestamps.append(timestamp)

# Convert lists to numpy arrays
classes_array = np.array(classes)
points_array = np.array(points)
timestamps_array = np.array(timestamps)

# Save the arrays as an NPZ file
np.savez(output_file, classes=classes_array, centerpoints=points_array, timestamps=timestamps_array)
print(f"Saved {len(points)} centerpoints to {output_file}")
print(points[:20])

class_names = [ x.champion.name for x in match.participants ]

class_file = "api_classes.txt"
with open(class_file, "w") as file:
    for class_name in class_names:
        file.write(class_name+'\n')


Saved 360 centerpoints to api_test.npz
[(554, 581), (593, 464), (557, 345), (458, 271), (335, 269), (14500, 14511), (14539, 14394), (14503, 14275), (14404, 14201), (14281, 14199), (1024, 3759), (7150, 3132), (5620, 6200), (7026, 3118), (10278, 3002), (6674, 11356), (11519, 7501), (8568, 6494), (13440, 4191), (13729, 3876)]
